### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', None)


<br>

### Main data

In [2]:
data = pd.read_csv('application_train.csv')
data.shape

(307511, 122)

In [3]:
data.head(1)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.0,0.069,0.0833,0.125,0.0369,0.0202,0.019,0.0,0.0,0.0252,0.0383,0.9722,0.6341,0.0144,0.0,0.069,0.0833,0.125,0.0377,0.022,0.0198,0.0,0.0,0.025,0.0369,0.9722,0.6243,0.0144,0.0,0.069,0.0833,0.125,0.0375,0.0205,0.0193,0.0,0.0,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0



<br>

<b>GENDER_CODE missing values:

In [4]:
# data[data.CODE_GENDER == 'XNA']

This gender category does not have enough diversity in target as it has all persons not defaulting plus there aren't enough samples to model them.

In [5]:
data = data[data.CODE_GENDER != 'XNA']


<br>

<B>OWN CAR AGE Missing values

In [6]:
data[data['OWN_CAR_AGE'].isna()]['FLAG_OWN_CAR'].value_counts()

N    202922
Y         5
Name: FLAG_OWN_CAR, dtype: int64

In [7]:
data[data.OWN_CAR_AGE.notnull()]['FLAG_OWN_CAR'].value_counts()

Y    104580
Name: FLAG_OWN_CAR, dtype: int64

Most of these do not have a car in the first place, so substituing a different value here.

In [8]:
data.loc[data.FLAG_OWN_CAR == 'N', 'OWN_CAR_AGE'] = -1

In [9]:
data.OWN_CAR_AGE.isna().sum()

5


<br>

<B>Creating new features:

In [10]:
avg_house = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',\
             'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG',\
             'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',\
             'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG']

mode_house = ['APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',\
              'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE',\
              'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE',\
              'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE']

medi_house = ['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', \
              'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI',\
              'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', \
              'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI' ]

flag_docs = [i for i in data.columns if 'FLAG_DOCUMENT' in i]

In [11]:
data['INCOME_PERCENT_CREDIT'] = data['AMT_INCOME_TOTAL']/data['AMT_CREDIT']

data['EMPLOYED_PERCENT_BIRTH'] = data['DAYS_EMPLOYED']/data['DAYS_BIRTH']

data['INCOME_DIST_FAM'] = data['AMT_INCOME_TOTAL']/data['CNT_FAM_MEMBERS']

data['INCOME_DIST_DEPENDENT'] = [i/j if (pd.Series([j]).notna()[0])&(j!=0) else 0 for i,j in zip(data['AMT_INCOME_TOTAL'],data['CNT_CHILDREN'])]

data['INCOME_PERC_ANNUITY'] = data['AMT_ANNUITY']/data['AMT_INCOME_TOTAL']

data['PAYMENT_RATE'] = data['AMT_ANNUITY']/data['AMT_CREDIT']

data['DEPENDENT_PERC'] = [i/j if pd.Series([j]).notna()[0] else 0 for i,j in zip(data['CNT_CHILDREN'], data['CNT_FAM_MEMBERS'])]

data['MEAN_AVG_HOUSING'] = data[avg_house].mean(axis=1)

data['MEAN_MEDI_HOUSING'] = data[medi_house].mean(axis=1)

data['MEAN_MODE_HOUSING'] = data[mode_house].mean(axis=1)

data['PERC_30_DEF_SOCIAL_CIRCLE'] = [i/j if (pd.Series([j]).notna()[0])&(j!=0) else 0 for i,j in zip(data['DEF_30_CNT_SOCIAL_CIRCLE'], data['OBS_30_CNT_SOCIAL_CIRCLE'])]

data['PERC_60_DEF_SOCIAL_CIRCLE'] = [i/j if (pd.Series([j]).notna()[0])&(j!=0) else 0 for i,j in zip(data['DEF_60_CNT_SOCIAL_CIRCLE'], data['OBS_60_CNT_SOCIAL_CIRCLE'])]

data['NO_OF_DOCS_SUBMITTED'] = data[flag_docs].sum(axis=1)

data['LIFE_PERC_OWNED_CAR'] = (-365.25*data['OWN_CAR_AGE'])/data['DAYS_BIRTH']

In [12]:
del avg_house; del medi_house; del mode_house; del flag_docs


<br>

<b>Apartment features:

In [14]:
apartments = [i for i in data.columns if ('AVG' in i)or('MEDI' in i)or('MODE' in i)]
# apartments[:10]

In [14]:
# data[apartments].corr()

In [15]:
del apartments

<br>


<b>Removing Median and Mode features as they are highly correlated with mean

In [16]:
rem = [i for i in data.columns if ('MEDI' in i)|('MODE' in i)]

for i in ['FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE',\
          'EMERGENCYSTATE_MODE', 'MEAN_MEDI_HOUSING', 'MEAN_MODE_HOUSING']:
    rem.remove(i)

In [17]:
data.drop(columns=rem, inplace=True)
del rem
data.shape

(307507, 108)

In [18]:
data.head(1)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,INCOME_PERCENT_CREDIT,EMPLOYED_PERCENT_BIRTH,INCOME_DIST_FAM,INCOME_DIST_DEPENDENT,INCOME_PERC_ANNUITY,PAYMENT_RATE,DEPENDENT_PERC,MEAN_AVG_HOUSING,MEAN_MEDI_HOUSING,MEAN_MODE_HOUSING,PERC_30_DEF_SOCIAL_CIRCLE,PERC_60_DEF_SOCIAL_CIRCLE,NO_OF_DOCS_SUBMITTED,LIFE_PERC_OWNED_CAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,-1.0,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.0,0.069,0.0833,0.125,0.0369,0.0202,0.019,0.0,0.0,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.498036,0.067329,202500.0,0.0,0.121978,0.060749,0.0,0.144336,0.144814,0.145786,1.0,1.0,1,-0.038606


In [19]:
data.drop(columns=['NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 
                   'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE'], inplace=True)


<br>

### Bureau & Bureau balance data

In [20]:
bureau = pd.read_csv('bureau.csv')
bureau.columns = [i+'_BUREAU' if 'SK_ID' not in i else i for i in bureau.columns ]
bur = pd.read_csv('bureau_balance.csv')
bur.columns = [i+'_BUREAU_BAL' if 'SK_ID' not in i else i for i in bur.columns ]
bureau.head(1)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE_BUREAU,CREDIT_CURRENCY_BUREAU,DAYS_CREDIT_BUREAU,CREDIT_DAY_OVERDUE_BUREAU,DAYS_CREDIT_ENDDATE_BUREAU,DAYS_ENDDATE_FACT_BUREAU,AMT_CREDIT_MAX_OVERDUE_BUREAU,CNT_CREDIT_PROLONG_BUREAU,AMT_CREDIT_SUM_BUREAU,AMT_CREDIT_SUM_DEBT_BUREAU,AMT_CREDIT_SUM_LIMIT_BUREAU,AMT_CREDIT_SUM_OVERDUE_BUREAU,CREDIT_TYPE_BUREAU,DAYS_CREDIT_UPDATE_BUREAU,AMT_ANNUITY_BUREAU
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN


In [21]:
bureau.shape

(1716428, 17)

In [22]:
# bureau['DEBT_RATE_BUREAU'] = [i/j if (pd.Series([j]).notna()[0])&(j != 0) else 0 for i,j in \
#                               zip(bureau['AMT_CREDIT_SUM_DEBT_BUREAU'].fillna(0), \
#                                   bureau['AMT_CREDIT_SUM_BUREAU'].fillna(1))]

# bureau['CREDIT_LIMIT_RATE_BUREAU'] = [i/j if (pd.Series([j]).notna()[0])&(j != 0) else 0 for i,j in \
#                               zip(bureau['AMT_CREDIT_SUM_LIMIT_BUREAU'].fillna(0), \
#                                   bureau['AMT_CREDIT_SUM_BUREAU'].fillna(1))]

<b>Missing values

In [23]:
# bureau.isna().sum()/len(bureau)

In [24]:
# bur.isna().sum()

<b>Unique values

In [25]:
# bur.nunique()

In [26]:
# bur.shape, bureau.shape

In [27]:
# len(bur.SK_ID_BUREAU.unique()), len(bureau.SK_ID_BUREAU.unique())

In [28]:
# len( set(bur.SK_ID_BUREAU.unique()).union(bureau.SK_ID_BUREAU.unique()) - set(bur.SK_ID_BUREAU.unique()).intersection(set(bureau.SK_ID_BUREAU.unique())) )

<b>Joining both

In [22]:
bureau = pd.merge(bureau, bur, how='inner', on='SK_ID_BUREAU')
bureau.head(1)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE_BUREAU,CREDIT_CURRENCY_BUREAU,DAYS_CREDIT_BUREAU,CREDIT_DAY_OVERDUE_BUREAU,DAYS_CREDIT_ENDDATE_BUREAU,DAYS_ENDDATE_FACT_BUREAU,AMT_CREDIT_MAX_OVERDUE_BUREAU,CNT_CREDIT_PROLONG_BUREAU,AMT_CREDIT_SUM_BUREAU,AMT_CREDIT_SUM_DEBT_BUREAU,AMT_CREDIT_SUM_LIMIT_BUREAU,AMT_CREDIT_SUM_OVERDUE_BUREAU,CREDIT_TYPE_BUREAU,DAYS_CREDIT_UPDATE_BUREAU,AMT_ANNUITY_BUREAU,MONTHS_BALANCE_BUREAU_BAL,STATUS_BUREAU_BAL
0,380361,5715448,Active,currency 1,-820,0,31069.0,NaN,NaN,0,67500.0,0.0,67500.0,0.0,Credit card,-183,0.0,0,C


In [23]:
del bur


<br>

### Functions to make functions by grouping by on ID variables

In [24]:
def aggr_numeric_cols(df, df_main, id_col):
    numeric = [i for i in  df.columns if (df[i].dtype != 'O')&('ID' not in i)]
    keep = ['SK_ID_CURR']+numeric
    kmin = [i if i.startswith('SK') else i+'_MIN' for i in keep]
    kmax = [i if i.startswith('SK') else i+'_MAX' for i in keep]
    kmean = [i if i.startswith('SK') else i+'_AVG' for i in keep]
    
    min_df = df.groupby(id_col, as_index=False)[numeric].min()
    min_df.columns = kmin
  
    max_df = df.groupby(id_col, as_index=False)[numeric].max()
    max_df.columns = kmax
    
    mean_df = df.groupby(id_col, as_index=False)[numeric].mean()
    mean_df.columns = kmean
    
    df_main = pd.merge(df_main, min_df, how='left', on=id_col)
    df_main = pd.merge(df_main, max_df, how='left', on=id_col)
    df_main = pd.merge(df_main, mean_df, how='left', on=id_col)
    
    return df_main

In [25]:
def aggr_categoric_cols(df, df_main, id_col):
    categoric = [i for i in  df.columns if df[i].dtype == 'O']
    df = pd.get_dummies(df[categoric])
    df[id_col] = df_main[id_col]
    df = df.groupby(id_col).sum()
    df_main = pd.merge(df_main, df, how='left', on=id_col)
    return df_main


<br>

### Using the function on bureau data

In [26]:
data = aggr_numeric_cols(bureau, data, 'SK_ID_CURR')

In [27]:
data = aggr_categoric_cols(bureau, data, 'SK_ID_CURR')

In [28]:
bureau.drop(columns=['SK_ID_BUREAU', 'CREDIT_ACTIVE_BUREAU', 'CREDIT_CURRENCY_BUREAU', 'DAYS_CREDIT_BUREAU',
                     'CREDIT_DAY_OVERDUE_BUREAU', 'DAYS_CREDIT_ENDDATE_BUREAU', 'DAYS_ENDDATE_FACT_BUREAU', 
                     'AMT_CREDIT_MAX_OVERDUE_BUREAU', 'CNT_CREDIT_PROLONG_BUREAU', 'AMT_CREDIT_SUM_OVERDUE_BUREAU',
                     'CREDIT_TYPE_BUREAU', 'DAYS_CREDIT_UPDATE_BUREAU', 'AMT_ANNUITY_BUREAU', 
                     'MONTHS_BALANCE_BUREAU_BAL', 'STATUS_BUREAU_BAL'], inplace=True)

In [61]:
data['AMT_CREDIT_SUM_BUREAU'] = data.SK_ID_CURR.map(bureau['AMT_CREDIT_SUM_BUREAU'])

data['AMT_CREDIT_SUM_DEBT_BUREAU'] = data.SK_ID_CURR.map(bureau['AMT_CREDIT_SUM_DEBT_BUREAU'])

data['AMT_CREDIT_SUM_LIMIT_BUREAU'] = data.SK_ID_CURR.map(bureau['AMT_CREDIT_SUM_LIMIT_BUREAU'])

In [62]:
print(data.shape)
data.head(2)

(307507, 173)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,INCOME_PERCENT_CREDIT,EMPLOYED_PERCENT_BIRTH,INCOME_DIST_FAM,INCOME_DIST_DEPENDENT,INCOME_PERC_ANNUITY,PAYMENT_RATE,DEPENDENT_PERC,MEAN_AVG_HOUSING,MEAN_MEDI_HOUSING,MEAN_MODE_HOUSING,PERC_30_DEF_SOCIAL_CIRCLE,PERC_60_DEF_SOCIAL_CIRCLE,NO_OF_DOCS_SUBMITTED,LIFE_PERC_OWNED_CAR,DAYS_CREDIT_BUREAU_MIN,CREDIT_DAY_OVERDUE_BUREAU_MIN,DAYS_CREDIT_ENDDATE_BUREAU_MIN,DAYS_ENDDATE_FACT_BUREAU_MIN,AMT_CREDIT_MAX_OVERDUE_BUREAU_MIN,CNT_CREDIT_PROLONG_BUREAU_MIN,AMT_CREDIT_SUM_BUREAU_MIN,AMT_CREDIT_SUM_DEBT_BUREAU_MIN,AMT_CREDIT_SUM_LIMIT_BUREAU_MIN,AMT_CREDIT_SUM_OVERDUE_BUREAU_MIN,DAYS_CREDIT_UPDATE_BUREAU_MIN,AMT_ANNUITY_BUREAU_MIN,MONTHS_BALANCE_BUREAU_BAL_MIN,DAYS_CREDIT_BUREAU_MAX,CREDIT_DAY_OVERDUE_BUREAU_MAX,DAYS_CREDIT_ENDDATE_BUREAU_MAX,DAYS_ENDDATE_FACT_BUREAU_MAX,AMT_CREDIT_MAX_OVERDUE_BUREAU_MAX,CNT_CREDIT_PROLONG_BUREAU_MAX,AMT_CREDIT_SUM_BUREAU_MAX,AMT_CREDIT_SUM_DEBT_BUREAU_MAX,AMT_CREDIT_SUM_LIMIT_BUREAU_MAX,AMT_CREDIT_SUM_OVERDUE_BUREAU_MAX,DAYS_CREDIT_UPDATE_BUREAU_MAX,AMT_ANNUITY_BUREAU_MAX,MONTHS_BALANCE_BUREAU_BAL_MAX,DAYS_CREDIT_BUREAU_AVG,CREDIT_DAY_OVERDUE_BUREAU_AVG,DAYS_CREDIT_ENDDATE_BUREAU_AVG,DAYS_ENDDATE_FACT_BUREAU_AVG,AMT_CREDIT_MAX_OVERDUE_BUREAU_AVG,CNT_CREDIT_PROLONG_BUREAU_AVG,AMT_CREDIT_SUM_BUREAU_AVG,AMT_CREDIT_SUM_DEBT_BUREAU_AVG,AMT_CREDIT_SUM_LIMIT_BUREAU_AVG,AMT_CREDIT_SUM_OVERDUE_BUREAU_AVG,DAYS_CREDIT_UPDATE_BUREAU_AVG,AMT_ANNUITY_BUREAU_AVG,MONTHS_BALANCE_BUREAU_BAL_AVG,CREDIT_ACTIVE_BUREAU_Active,CREDIT_ACTIVE_BUREAU_Bad debt,CREDIT_ACTIVE_BUREAU_Closed,CREDIT_ACTIVE_BUREAU_Sold,CREDIT_CURRENCY_BUREAU_currency 1,CREDIT_CURRENCY_BUREAU_currency 2,CREDIT_CURRENCY_BUREAU_currency 3,CREDIT_CURRENCY_BUREAU_currency 4,CREDIT_TYPE_BUREAU_Another type of loan,CREDIT_TYPE_BUREAU_Car loan,CREDIT_TYPE_BUREAU_Cash loan (non-earmarked),CREDIT_TYPE_BUREAU_Consumer credit,CREDIT_TYPE_BUREAU_Credit card,CREDIT_TYPE_BUREAU_Loan for business development,CREDIT_TYPE_BUREAU_Loan for purchase of shares (margin lending),CREDIT_TYPE_BUREAU_Loan for the purchase of equipment,CREDIT_TYPE_BUREAU_Loan for working capital replenishment,CREDIT_TYPE_BUREAU_Microloan,CREDIT_TYPE_BUREAU_Mobile operator loan,CREDIT_TYPE_BUREAU_Mortgage,CREDIT_TYPE_BUREAU_Real estate loan,CREDIT_TYPE_BUREAU_Unknown type of loan,STATUS_BUREAU_BAL_0,STATUS_BUREAU_BAL_1,STATUS_BUREAU_BAL_2,STATUS_BUREAU_BAL_3,STATUS_BUREAU_BAL_4,STATUS_BUREAU_BAL_5,STATUS_BUREAU_BAL_C,STATUS_BUREAU_BAL_X,AMT_CREDIT_SUM_BUREAU,A

In [63]:
del bureau

In [64]:
data['DEBT_RATE_BUREAU'] = [i/j if (pd.Series([j]).notna()[0])&(j != 0) else 0 for i,j in \
                              zip(data['AMT_CREDIT_SUM_DEBT_BUREAU'].fillna(0), \
                                  data['AMT_CREDIT_SUM_BUREAU'].fillna(1))]

data['CREDIT_LIMIT_RATE_BUREAU'] = [i/j if (pd.Series([j]).notna()[0])&(j != 0) else 0 for i,j in \
                              zip(data['AMT_CREDIT_SUM_LIMIT_BUREAU'].fillna(0), \
                                  data['AMT_CREDIT_SUM_BUREAU'].fillna(1))]

In [66]:
data.isna().sum()/len(data)

SK_ID_CURR                                                         0.000000
TARGET                                                             0.000000
NAME_CONTRACT_TYPE                                                 0.000000
CODE_GENDER                                                        0.000000
FLAG_OWN_CAR                                                       0.000000
FLAG_OWN_REALTY                                                    0.000000
CNT_CHILDREN                                                       0.000000
AMT_INCOME_TOTAL                                                   0.000000
AMT_CREDIT                                                         0.000000
AMT_ANNUITY                                                        0.000039
AMT_GOODS_PRICE                                                    0.000904
NAME_TYPE_SUITE                                                    0.004202
NAME_INCOME_TYPE                                                   0.000000
NAME_EDUCATI

In [70]:
del ps

In [71]:
## How to check stored variables in memory
# import sys

# local_vars = list(locals().items())
# for var, obj in local_vars:
#     print(var, sys.getsizeof(obj))


<br>

### POS cash balance data

In [72]:
pos_cash = pd.read_csv('POS_CASH_balance.csv')
pos_cash.drop(columns='SK_ID_PREV', inplace=True)
pos_cash.columns = [i+'_POS' if 'SK_ID' not in i else i for i in pos_cash.columns]
pos_cash.shape

(10001358, 7)

In [73]:
pos_cash.head()

,SK_ID_CURR,MONTHS_BALANCE_POS,CNT_INSTALMENT_POS,CNT_INSTALMENT_FUTURE_POS,NAME_CONTRACT_STATUS_POS,SK_DPD_POS,SK_DPD_DEF_POS
0,182943,-31,48.0,45.0,Active,0,0
1,367990,-33,36.0,35.0,Active,0,0
2,397406,-32,12.0,9.0,Active,0,0
3,269225,-35,48.0,42.0,Active,0,0
4,334279,-35,36.0,35.0,Active,0,0


In [74]:
pos_cash['PERC_INSTAL_PAID_POS'] = (pos_cash['CNT_INSTALMENT_POS'] - pos_cash['CNT_INSTALMENT_FUTURE_POS'])/pos_cash['CNT_INSTALMENT_POS']

<b>Applying functions on pos cash

In [77]:
data = aggr_numeric_cols(pos_cash, data, 'SK_ID_CURR')

In [78]:
data = aggr_categoric_cols(pos_cash, data, 'SK_ID_CURR')

In [79]:
data.shape

(307507, 199)

In [80]:
del pos_cash

In [82]:
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,INCOME_PERCENT_CREDIT,EMPLOYED_PERCENT_BIRTH,INCOME_DIST_FAM,INCOME_DIST_DEPENDENT,INCOME_PERC_ANNUITY,PAYMENT_RATE,DEPENDENT_PERC,MEAN_AVG_HOUSING,MEAN_MEDI_HOUSING,MEAN_MODE_HOUSING,PERC_30_DEF_SOCIAL_CIRCLE,PERC_60_DEF_SOCIAL_CIRCLE,NO_OF_DOCS_SUBMITTED,LIFE_PERC_OWNED_CAR,DAYS_CREDIT_BUREAU_MIN,CREDIT_DAY_OVERDUE_BUREAU_MIN,DAYS_CREDIT_ENDDATE_BUREAU_MIN,DAYS_ENDDATE_FACT_BUREAU_MIN,AMT_CREDIT_MAX_OVERDUE_BUREAU_MIN,CNT_CREDIT_PROLONG_BUREAU_MIN,AMT_CREDIT_SUM_BUREAU_MIN,AMT_CREDIT_SUM_DEBT_BUREAU_MIN,AMT_CREDIT_SUM_LIMIT_BUREAU_MIN,AMT_CREDIT_SUM_OVERDUE_BUREAU_MIN,DAYS_CREDIT_UPDATE_BUREAU_MIN,AMT_ANNUITY_BUREAU_MIN,MONTHS_BALANCE_BUREAU_BAL_MIN,DAYS_CREDIT_BUREAU_MAX,CREDIT_DAY_OVERDUE_BUREAU_MAX,DAYS_CREDIT_ENDDATE_BUREAU_MAX,DAYS_ENDDATE_FACT_BUREAU_MAX,AMT_CREDIT_MAX_OVERDUE_BUREAU_MAX,CNT_CREDIT_PROLONG_BUREAU_MAX,AMT_CREDIT_SUM_BUREAU_MAX,AMT_CREDIT_SUM_DEBT_BUREAU_MAX,AMT_CREDIT_SUM_LIMIT_BUREAU_MAX,AMT_CREDIT_SUM_OVERDUE_BUREAU_MAX,DAYS_CREDIT_UPDATE_BUREAU_MAX,AMT_ANNUITY_BUREAU_MAX,MONTHS_BALANCE_BUREAU_BAL_MAX,DAYS_CREDIT_BUREAU_AVG,CREDIT_DAY_OVERDUE_BUREAU_AVG,DAYS_CREDIT_ENDDATE_BUREAU_AVG,DAYS_ENDDATE_FACT_BUREAU_AVG,AMT_CREDIT_MAX_OVERDUE_BUREAU_AVG,CNT_CREDIT_PROLONG_BUREAU_AVG,AMT_CREDIT_SUM_BUREAU_AVG,AMT_CREDIT_SUM_DEBT_BUREAU_AVG,AMT_CREDIT_SUM_LIMIT_BUREAU_AVG,AMT_CREDIT_SUM_OVERDUE_BUREAU_AVG,DAYS_CREDIT_UPDATE_BUREAU_AVG,AMT_ANNUITY_BUREAU_AVG,MONTHS_BALANCE_BUREAU_BAL_AVG,CREDIT_ACTIVE_BUREAU_Active,CREDIT_ACTIVE_BUREAU_Bad debt,CREDIT_ACTIVE_BUREAU_Closed,CREDIT_ACTIVE_BUREAU_Sold,CREDIT_CURRENCY_BUREAU_currency 1,CREDIT_CURRENCY_BUREAU_currency 2,CREDIT_CURRENCY_BUREAU_currency 3,CREDIT_CURRENCY_BUREAU_currency 4,CREDIT_TYPE_BUREAU_Another type of loan,CREDIT_TYPE_BUREAU_Car loan,CREDIT_TYPE_BUREAU_Cash loan (non-earmarked),CREDIT_TYPE_BUREAU_Consumer credit,CREDIT_TYPE_BUREAU_Credit card,CREDIT_TYPE_BUREAU_Loan for business development,CREDIT_TYPE_BUREAU_Loan for purchase of shares (margin lending),CREDIT_TYPE_BUREAU_Loan for the purchase of equipment,CREDIT_TYPE_BUREAU_Loan for working capital replenishment,CREDIT_TYPE_BUREAU_Microloan,CREDIT_TYPE_BUREAU_Mobile operator loan,CREDIT_TYPE_BUREAU_Mortgage,CREDIT_TYPE_BUREAU_Real estate loan,CREDIT_TYPE_BUREAU_Unknown type of loan,STATUS_BUREAU_BAL_0,STATUS_BUREAU_BAL_1,STATUS_BUREAU_BAL_2,STATUS_BUREAU_BAL_3,STATUS_BUREAU_BAL_4,STATUS_BUREAU_BAL_5,STATUS_BUREAU_BAL_C,STATUS_BUREAU_BAL_X,AMT_CREDIT_SUM_BUREAU,A


<br>

### Credit card balance data

In [81]:
credit_card = pd.read_csv('credit_card_balance.csv')
credit_card.drop(columns='SK_ID_PREV', inplace=True)
credit_card.columns = [i+'_CREDIT' if 'SK_ID' not in i else i for i in credit_card.columns]
print(credit_card.shape)
credit_card.head()

(3840312, 22)


,SK_ID_CURR,MONTHS_BALANCE_CREDIT,AMT_BALANCE_CREDIT,AMT_CREDIT_LIMIT_ACTUAL_CREDIT,AMT_DRAWINGS_ATM_CURRENT_CREDIT,AMT_DRAWINGS_CURRENT_CREDIT,AMT_DRAWINGS_OTHER_CURRENT_CREDIT,AMT_DRAWINGS_POS_CURRENT_CREDIT,AMT_INST_MIN_REGULARITY_CREDIT,AMT_PAYMENT_CURRENT_CREDIT,AMT_PAYMENT_TOTAL_CURRENT_CREDIT,AMT_RECEIVABLE_PRINCIPAL_CREDIT,AMT_RECIVABLE_CREDIT,AMT_TOTAL_RECEIVABLE_CREDIT,CNT_DRAWINGS_ATM_CURRENT_CREDIT,CNT_DRAWINGS_CURRENT_CREDIT,CNT_DRAWINGS_OTHER_CURRENT_CREDIT,CNT_DRAWINGS_POS_CURRENT_CREDIT,CNT_INSTALMENT_MATURE_CUM_CREDIT,NAME_CONTRACT_STATUS_CREDIT,SK_DPD_CREDIT,SK_DPD_DEF_CREDIT
0,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [83]:
credit_card['EXPENSE_PERC_CREDIT'] = \
(credit_card['AMT_CREDIT_LIMIT_ACTUAL_CREDIT'] - credit_card['AMT_BALANCE_CREDIT'])/\
credit_card['AMT_CREDIT_LIMIT_ACTUAL_CREDIT']

<b>Applying the functions on CCB data
 

In [86]:
data = aggr_numeric_cols(credit_card, data, 'SK_ID_CURR')

In [87]:
data = aggr_categoric_cols(credit_card, data, 'SK_ID_CURR')

In [88]:
data.shape

(307507, 269)

In [89]:
del credit_card


<br>

### Previous application data

In [90]:
previous = pd.read_csv('previous_application.csv')
previous.drop(columns='SK_ID_PREV', inplace=True)
previous.columns = [i+'_PREV' if 'SK' not in i else i for i in previous.columns]
previous.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_PREV,AMT_ANNUITY_PREV,AMT_APPLICATION_PREV,AMT_CREDIT_PREV,AMT_DOWN_PAYMENT_PREV,AMT_GOODS_PRICE_PREV,WEEKDAY_APPR_PROCESS_START_PREV,HOUR_APPR_PROCESS_START_PREV,FLAG_LAST_APPL_PER_CONTRACT_PREV,NFLAG_LAST_APPL_IN_DAY_PREV,RATE_DOWN_PAYMENT_PREV,RATE_INTEREST_PRIMARY_PREV,RATE_INTEREST_PRIVILEGED_PREV,NAME_CASH_LOAN_PURPOSE_PREV,NAME_CONTRACT_STATUS_PREV,DAYS_DECISION_PREV,NAME_PAYMENT_TYPE_PREV,CODE_REJECT_REASON_PREV,NAME_TYPE_SUITE_PREV,NAME_CLIENT_TYPE_PREV,NAME_GOODS_CATEGORY_PREV,NAME_PORTFOLIO_PREV,NAME_PRODUCT_TYPE_PREV,CHANNEL_TYPE_PREV,SELLERPLACE_AREA_PREV,NAME_SELLER_INDUSTRY_PREV,CNT_PAYMENT_PREV,NAME_YIELD_GROUP_PREV,PRODUCT_COMBINATION_PREV,DAYS_FIRST_DRAWING_PREV,DAYS_FIRST_DUE_PREV,DAYS_LAST_DUE_1ST_VERSION_PREV,DAYS_LAST_DUE_PREV,DAYS_TERMINATION_PREV,NFLAG_INSURED_ON_APPROVAL_PREV
0,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
previous[previous['AMT_APPLICATION_PREV'] == 0].shape

(392402, 36)

In [ ]:
# previous['SELLERPLACE_AREA_PREV'] = previous['SELLERPLACE_AREA_PREV'].astype('O')

In [96]:
previous['PAYMENT_RATE_PREV'] = [i/j if (pd.Series([j]).notna()[0])&(j!=0) else 0 for i,j in \
                                 zip(previous['AMT_ANNUITY_PREV'],previous['AMT_CREDIT_PREV'])]

previous['APPROVED_PERC_PREV'] = [i/j if (pd.Series([j]).notna()[0])&(j!=0) else 0 for i,j in \
                                  zip(previous['AMT_CREDIT_PREV'], previous['AMT_APPLICATION_PREV'])]

previous['PAYMENT_RATE_PREV'] = [i/j if (pd.Series([j]).notna()[0])&(j!=0) else 0 for i,j in \
                                 zip(previous['AMT_DOWN_PAYMENT_PREV'], previous['AMT_CREDIT_PREV'])]

<b>Applying the functions

In [97]:
data = aggr_numeric_cols(previous, data, 'SK_ID_CURR')
data.shape

(307507, 332)

In [98]:
data = aggr_categoric_cols(previous, data, 'SK_ID_CURR')
data.shape

(307507, 475)

In [99]:
del previous

In [100]:
data.head(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,INCOME_PERCENT_CREDIT,EMPLOYED_PERCENT_BIRTH,INCOME_DIST_FAM,INCOME_DIST_DEPENDENT,INCOME_PERC_ANNUITY,PAYMENT_RATE,DEPENDENT_PERC,MEAN_AVG_HOUSING,MEAN_MEDI_HOUSING,MEAN_MODE_HOUSING,PERC_30_DEF_SOCIAL_CIRCLE,PERC_60_DEF_SOCIAL_CIRCLE,NO_OF_DOCS_SUBMITTED,LIFE_PERC_OWNED_CAR,DAYS_CREDIT_BUREAU_MIN,CREDIT_DAY_OVERDUE_BUREAU_MIN,DAYS_CREDIT_ENDDATE_BUREAU_MIN,DAYS_ENDDATE_FACT_BUREAU_MIN,AMT_CREDIT_MAX_OVERDUE_BUREAU_MIN,CNT_CREDIT_PROLONG_BUREAU_MIN,AMT_CREDIT_SUM_BUREAU_MIN,AMT_CREDIT_SUM_DEBT_BUREAU_MIN,AMT_CREDIT_SUM_LIMIT_BUREAU_MIN,AMT_CREDIT_SUM_OVERDUE_BUREAU_MIN,DAYS_CREDIT_UPDATE_BUREAU_MIN,AMT_ANNUITY_BUREAU_MIN,MONTHS_BALANCE_BUREAU_BAL_MIN,DAYS_CREDIT_BUREAU_MAX,CREDIT_DAY_OVERDUE_BUREAU_MAX,DAYS_CREDIT_ENDDATE_BUREAU_MAX,DAYS_ENDDATE_FACT_BUREAU_MAX,AMT_CREDIT_MAX_OVERDUE_BUREAU_MAX,CNT_CREDIT_PROLONG_BUREAU_MAX,AMT_CREDIT_SUM_BUREAU_MAX,AMT_CREDIT_SUM_DEBT_BUREAU_MAX,AMT_CREDIT_SUM_LIMIT_BUREAU_MAX,AMT_CREDIT_SUM_OVERDUE_BUREAU_MAX,DAYS_CREDIT_UPDATE_BUREAU_MAX,...,NAME_CASH_LOAN_PURPOSE_PREV_Buying a new car,NAME_CASH_LOAN_PURPOSE_PREV_Buying a used car,NAME_CASH_LOAN_PURPOSE_PREV_Car repairs,NAME_CASH_LOAN_PURPOSE_PREV_Education,NAME_CASH_LOAN_PURPOSE_PREV_Everyday expenses,NAME_CASH_LOAN_PURPOSE_PREV_Furniture,NAME_CASH_LOAN_PURPOSE_PREV_Gasification / water supply,NAME_CASH_LOAN_PURPOSE_PREV_Hobby,NAME_CASH_LOAN_PURPOSE_PREV_Journey,NAME_CASH_LOAN_PURPOSE_PREV_Medicine,NAME_CASH_LOAN_PURPOSE_PREV_Money for a third person,NAME_CASH_LOAN_PURPOSE_PREV_Other,NAME_CASH_LOAN_PURPOSE_PREV_Payments on other loans,NAME_CASH_LOAN_PURPOSE_PREV_Purchase of electronic equipment,NAME_CASH_LOAN_PURPOSE_PREV_Refusal to name the goal,NAME_CASH_LOAN_PURPOSE_PREV_Repairs,NAME_CASH_LOAN_PURPOSE_PREV_Urgent needs,NAME_CASH_LOAN_PURPOSE_PREV_Wedding / gift / holiday,NAME_CASH_LOAN_PURPOSE_PREV_XAP,NAME_CASH_LOAN_PURPOSE_PREV_XNA,NAME_CONTRACT_STATUS_PREV_Approved,NAME_CONTRACT_STATUS_PREV_Canceled,NAME_CONTRACT_STATUS_PREV_Refused,NAME_CONTRACT_STATUS_PREV_Unused offer,NAME_PAYMENT_TYPE_PREV_Cash through the bank,NAME_PAYMENT_TYPE_PREV_Cashless from the account of the employer,NAME_PAYMENT_TYPE_PREV_Non-cash from your account,NAME_PAYMENT_TYPE_PREV_XNA,CODE_REJECT_REASON_PREV_CLIENT,CODE_REJECT_REASON_PREV_HC,CODE_REJECT_REASON_PREV_LIMIT,CODE_REJECT_REASON_PREV_SCO,CODE_REJECT_REASON_PREV_SCOFR,CODE_REJECT_REASON_PREV_SYSTEM,CODE_REJECT_REASON_PREV_VERIF,CODE_REJECT_REASON_PREV_XAP,CODE_REJECT_


<br>

### Instalment payment data

In [101]:
instalments = pd.read_csv('installments_payments.csv')
instalments.drop(columns='SK_ID_PREV', inplace=True)
instalments.columns = [i+'_INSTAL' if 'SK' not in i else i for i in instalments.columns]
instalments.shape

(13605401, 7)

In [102]:
instalments.head(2)

,SK_ID_CURR,NUM_INSTALMENT_VERSION_INSTAL,NUM_INSTALMENT_NUMBER_INSTAL,DAYS_INSTALMENT_INSTAL,DAYS_ENTRY_PAYMENT_INSTAL,AMT_INSTALMENT_INSTAL,AMT_PAYMENT_INSTAL
0,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525


In [103]:
instalments['NUM_INSTALMENT_VERSION_INSTAL'].nunique()

65

In [104]:
instalments['NUM_INSTALMENT_VERSION_INSTAL'] = \
instalments['NUM_INSTALMENT_VERSION_INSTAL'].astype('O')


<br>

<b>Applying the numeric function

In [105]:
data = aggr_numeric_cols(instalments, data, 'SK_ID_CURR')
data.shape

(307507, 490)

In [106]:
del instalments

 
 <br>

### Checking the Dataframe

substitute denominator in income dist dependent by 1 if it is zero

In [115]:
print(data.shape)
data.head(5)

(307507, 271)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,LANDAREA_AVG,NONLIVINGAREA_AVG,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,INCOME_PERC_ANNUITY,PAYMENT_RATE,NO_OF_DOCS_SUBMITTED,CREDIT_ACTIVE_BUREAU_Active,CREDIT_ACTIVE_BUREAU_Bad debt,CREDIT_ACTIVE_BUREAU_Sold,CREDIT_CURRENCY_BUREAU_currency 1,CREDIT_CURRENCY_BUREAU_currency 2,CREDIT_CURRENCY_BUREAU_currency 4,CREDIT_TYPE_BUREAU_Another type of loan,CREDIT_TYPE_BUREAU_Car loan,CREDIT_TYPE_BUREAU_Cash loan (non-earmarked),CREDIT_TYPE_BUREAU_Consumer credit,CREDIT_TYPE_BUREAU_Loan for business development,CREDIT_TYPE_BUREAU_Loan for purchase of shares (margin lending),CREDIT_TYPE_BUREAU_Loan for the purchase of equipment,CREDIT_TYPE_BUREAU_Loan for working capital replenishment,CREDIT_TYPE_BUREAU_Microloan,CREDIT_TYPE_BUREAU_Mobile operator loan,CREDIT_TYPE_BUREAU_Mortgage,CREDIT_TYPE_BUREAU_Real estate loan,CREDIT_TYPE_BUREAU_Unknown type of loan,STATUS_BUREAU_BAL_0,STATUS_BUREAU_BAL_1,STATUS_BUREAU_BAL_2,STATUS_BUREAU_BAL_3,STATUS_BUREAU_BAL_4,STATUS_BUREAU_BAL_5,STATUS_BUREAU_BAL_C,STATUS_BUREAU_BAL_X,AMT_CREDIT_SUM_BUREAU,AMT_CREDIT_SUM_LIMIT_BUREAU,DEBT_RATE_BUREAU,CREDIT_LIMIT_RATE_BUREAU,MONTHS_BALANCE_POS_MIN,CNT_INSTALMENT_POS_MIN,CNT_INSTALMENT_FUTURE_POS_MIN,SK_DPD_POS_x,SK_DPD_DEF_POS_x,MONTHS_BALANCE_POS_MAX,CNT_INSTALMENT_POS_MAX,SK_DPD_POS_y,SK_DPD_DEF_POS_y,NAME_CONTRACT_STATUS_POS_Active,NAME_CONTRACT_STATUS_POS_Amortized debt,NAME_CONTRACT_STATUS_POS_Approved,NAME_CONTRACT_STATUS_POS_Canceled,NAME_CONTRACT_STATUS_POS_Demand,NAME_CONTRACT_STATUS_POS_Returned to the store,NAME_CONTRACT_STATUS_POS_Signed,NAME_CONTRACT_STATUS_POS_XNA,NAME_CONTRACT_STATUS_CREDIT_Active,NAME_CONTRACT_STATUS_CREDIT_Approved,NAME_CONTRACT_STATUS_CREDIT_Demand,NAME_CONTRACT_STATUS_CREDIT_Refused,...,SELLERPLACE_AREA_PREV_MAX,DAYS_FIRST_DUE_PREV_MAX,DAYS_LAST_DUE_PREV_MAX,NFLAG_INSURED_ON_APPROVAL_PREV_MAX,APPROVED_PERC_PREV_MAX,NAME_CONTRACT_TYPE_PREV_Cash loans,NAME_CONTRACT_TYPE_PREV_Revolving loans,NAME_CONTRACT_TYPE_PREV_XNA,WEEKDAY_APPR_PROCESS_START_PREV_FRIDAY,WEEKDAY_APPR_PROCESS_START_PREV_MONDAY,WEEKDAY_APPR_PROCESS_START_PREV_SATURDAY,WEEKDAY_APPR_PROCESS_START_PREV_SUNDAY,WEEKDAY_APPR_PROCESS_START_PREV_THURSDAY,WEEKDAY_APPR_PROCESS_START_PREV_TUESDAY,WEEKDAY_APPR_PROCESS_START_PREV_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_PREV_N,NAME_CASH_LOAN_PURPOSE_PREV_Building a house or an annex,NAME_CASH_LOAN_PURPOSE_PREV_Business development,NAME_CASH_LOAN_PURPOSE_PREV_Buying a garage,NAME_CASH_LOAN_PURPOSE_PREV_Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE_PREV_Buying a home,NAME_CASH_LOAN_PURPOSE_PREV_Buying a new car,NAME_CASH_LOAN_PURPOSE_PREV_Buying a used car,NAME_CASH_LOAN_PURPOSE_PREV_Car repairs,NAME_CASH_LOAN_PURPOSE_PREV_Education,NAME_CASH_LOAN_PURPOSE_PREV_Everyday expenses,NAME_CASH_LOAN_PURPOSE_PREV_Furniture,NAME_CASH_LOAN_PURPOSE_PREV_Gasification / water supply,NA

In [108]:
data.isna().sum()/len(data)

SK_ID_CURR                                                          0.000000
TARGET                                                              0.000000
NAME_CONTRACT_TYPE                                                  0.000000
CODE_GENDER                                                         0.000000
FLAG_OWN_CAR                                                        0.000000
FLAG_OWN_REALTY                                                     0.000000
CNT_CHILDREN                                                        0.000000
AMT_INCOME_TOTAL                                                    0.000000
AMT_CREDIT                                                          0.000000
AMT_ANNUITY                                                         0.000039
AMT_GOODS_PRICE                                                     0.000904
NAME_TYPE_SUITE                                                     0.004202
NAME_INCOME_TYPE                                                    0.000000


<br>

### Removing variables with too many missing values

In [109]:
b = data.isna().sum()/len(data)
rem = b[b > 0.65].index.tolist()
rem

['YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'FLOORSMIN_AVG',
 'LIVINGAPARTMENTS_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'DAYS_CREDIT_BUREAU_MIN',
 'CREDIT_DAY_OVERDUE_BUREAU_MIN',
 'DAYS_CREDIT_ENDDATE_BUREAU_MIN',
 'DAYS_ENDDATE_FACT_BUREAU_MIN',
 'AMT_CREDIT_MAX_OVERDUE_BUREAU_MIN',
 'CNT_CREDIT_PROLONG_BUREAU_MIN',
 'AMT_CREDIT_SUM_BUREAU_MIN',
 'AMT_CREDIT_SUM_DEBT_BUREAU_MIN',
 'AMT_CREDIT_SUM_LIMIT_BUREAU_MIN',
 'AMT_CREDIT_SUM_OVERDUE_BUREAU_MIN',
 'DAYS_CREDIT_UPDATE_BUREAU_MIN',
 'AMT_ANNUITY_BUREAU_MIN',
 'MONTHS_BALANCE_BUREAU_BAL_MIN',
 'DAYS_CREDIT_BUREAU_MAX',
 'CREDIT_DAY_OVERDUE_BUREAU_MAX',
 'DAYS_CREDIT_ENDDATE_BUREAU_MAX',
 'DAYS_ENDDATE_FACT_BUREAU_MAX',
 'AMT_CREDIT_MAX_OVERDUE_BUREAU_MAX',
 'CNT_CREDIT_PROLONG_BUREAU_MAX',
 'AMT_CREDIT_SUM_BUREAU_MAX',
 'AMT_CREDIT_SUM_DEBT_BUREAU_MAX',
 'AMT_CREDIT_SUM_LIMIT_BUREAU_MAX',
 'AMT_CREDIT_SUM_OVERDUE_BUREAU_MAX',
 'DAYS_CREDIT_UPDATE_BUREAU_MAX',
 'AMT_ANNUITY_BUREAU_MAX',
 'MONTHS_BALANCE_BUREAU_BAL_MAX',
 'DAYS_CREDIT_BUREAU

In [110]:
len(rem)

113

In [111]:
data.drop(columns=rem, inplace=True)
data.shape

(307507, 377)


<br>

### Removing highly correlated variables

In [113]:
def remove_corr_vars(df):
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find features with correlation greater than 0.9
    to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]
    print(to_drop)
    
    try:
        to_drop.remove('TARGET')
    except:
        print('Target variable is not in the list.')

    # Drop features 
    df.drop(to_drop, axis=1, inplace=True)
    
    return df

data = remove_corr_vars(data)

['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'FLAG_EMP_PHONE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT_W_CITY', 'LIVE_REGION_NOT_WORK_REGION', 'LIVE_CITY_NOT_WORK_CITY', 'ELEVATORS_AVG', 'LIVINGAREA_AVG', 'TOTALAREA_MODE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'INCOME_PERCENT_CREDIT', 'EMPLOYED_PERCENT_BIRTH', 'INCOME_DIST_FAM', 'INCOME_DIST_DEPENDENT', 'DEPENDENT_PERC', 'MEAN_AVG_HOUSING', 'MEAN_MEDI_HOUSING', 'MEAN_MODE_HOUSING', 'PERC_30_DEF_SOCIAL_CIRCLE', 'PERC_60_DEF_SOCIAL_CIRCLE', 'LIFE_PERC_OWNED_CAR', 'CREDIT_ACTIVE_BUREAU_Closed', 'CREDIT_CURRENCY_BUREAU_currency 3', 'CREDIT_TYPE_BUREAU_Credit card', 'AMT_CREDIT_SUM_DEBT_BUREAU', 'CNT_INSTALMENT_FUTURE_POS_MAX', 'MONTHS_BALANCE_POS_AVG', 'CNT_INSTALMENT_POS_AVG', 'CNT_INSTALMENT_FUTURE_POS_AVG', 'SK_DPD_POS', 'SK_DPD_DEF_POS', 'NAME_CONTRACT_STATUS_POS_Completed', 'NAME_CONTRACT_STATUS_CREDIT_Completed', 'AMT_CREDIT_PREV_MIN', 'AMT_GOODS_PRICE_PREV_MIN', 'DAYS_DECISION_PREV_MIN', 'DAYS_TERMINATION_PREV_MIN', 'PAYMEN

In [114]:
data.shape

(307507, 271)

 
 <br>

### Viewing the final data

In [116]:
data.head(20)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,LANDAREA_AVG,NONLIVINGAREA_AVG,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,INCOME_PERC_ANNUITY,PAYMENT_RATE,NO_OF_DOCS_SUBMITTED,CREDIT_ACTIVE_BUREAU_Active,CREDIT_ACTIVE_BUREAU_Bad debt,CREDIT_ACTIVE_BUREAU_Sold,CREDIT_CURRENCY_BUREAU_currency 1,CREDIT_CURRENCY_BUREAU_currency 2,CREDIT_CURRENCY_BUREAU_currency 4,CREDIT_TYPE_BUREAU_Another type of loan,CREDIT_TYPE_BUREAU_Car loan,CREDIT_TYPE_BUREAU_Cash loan (non-earmarked),CREDIT_TYPE_BUREAU_Consumer credit,CREDIT_TYPE_BUREAU_Loan for business development,CREDIT_TYPE_BUREAU_Loan for purchase of shares (margin lending),CREDIT_TYPE_BUREAU_Loan for the purchase of equipment,CREDIT_TYPE_BUREAU_Loan for working capital replenishment,CREDIT_TYPE_BUREAU_Microloan,CREDIT_TYPE_BUREAU_Mobile operator loan,CREDIT_TYPE_BUREAU_Mortgage,CREDIT_TYPE_BUREAU_Real estate loan,CREDIT_TYPE_BUREAU_Unknown type of loan,STATUS_BUREAU_BAL_0,STATUS_BUREAU_BAL_1,STATUS_BUREAU_BAL_2,STATUS_BUREAU_BAL_3,STATUS_BUREAU_BAL_4,STATUS_BUREAU_BAL_5,STATUS_BUREAU_BAL_C,STATUS_BUREAU_BAL_X,AMT_CREDIT_SUM_BUREAU,AMT_CREDIT_SUM_LIMIT_BUREAU,DEBT_RATE_BUREAU,CREDIT_LIMIT_RATE_BUREAU,MONTHS_BALANCE_POS_MIN,CNT_INSTALMENT_POS_MIN,CNT_INSTALMENT_FUTURE_POS_MIN,SK_DPD_POS_x,SK_DPD_DEF_POS_x,MONTHS_BALANCE_POS_MAX,CNT_INSTALMENT_POS_MAX,SK_DPD_POS_y,SK_DPD_DEF_POS_y,NAME_CONTRACT_STATUS_POS_Active,NAME_CONTRACT_STATUS_POS_Amortized debt,NAME_CONTRACT_STATUS_POS_Approved,NAME_CONTRACT_STATUS_POS_Canceled,NAME_CONTRACT_STATUS_POS_Demand,NAME_CONTRACT_STATUS_POS_Returned to the store,NAME_CONTRACT_STATUS_POS_Signed,NAME_CONTRACT_STATUS_POS_XNA,NAME_CONTRACT_STATUS_CREDIT_Active,NAME_CONTRACT_STATUS_CREDIT_Approved,NAME_CONTRACT_STATUS_CREDIT_Demand,NAME_CONTRACT_STATUS_CREDIT_Refused,...,SELLERPLACE_AREA_PREV_MAX,DAYS_FIRST_DUE_PREV_MAX,DAYS_LAST_DUE_PREV_MAX,NFLAG_INSURED_ON_APPROVAL_PREV_MAX,APPROVED_PERC_PREV_MAX,NAME_CONTRACT_TYPE_PREV_Cash loans,NAME_CONTRACT_TYPE_PREV_Revolving loans,NAME_CONTRACT_TYPE_PREV_XNA,WEEKDAY_APPR_PROCESS_START_PREV_FRIDAY,WEEKDAY_APPR_PROCESS_START_PREV_MONDAY,WEEKDAY_APPR_PROCESS_START_PREV_SATURDAY,WEEKDAY_APPR_PROCESS_START_PREV_SUNDAY,WEEKDAY_APPR_PROCESS_START_PREV_THURSDAY,WEEKDAY_APPR_PROCESS_START_PREV_TUESDAY,WEEKDAY_APPR_PROCESS_START_PREV_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_PREV_N,NAME_CASH_LOAN_PURPOSE_PREV_Building a house or an annex,NAME_CASH_LOAN_PURPOSE_PREV_Business development,NAME_CASH_LOAN_PURPOSE_PREV_Buying a garage,NAME_CASH_LOAN_PURPOSE_PREV_Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE_PREV_Buying a home,NAME_CASH_LOAN_PURPOSE_PREV_Buying a new car,NAME_CASH_LOAN_PURPOSE_PREV_Buying a used car,NAME_CASH_LOAN_PURPOSE_PREV_Car repairs,NAME_CASH_LOAN_PURPOSE_PREV_Education,NAME_CASH_LOAN_PURPOSE_PREV_Everyday expenses,NAME_CASH_LOAN_PURPOSE_PREV_Furniture,NAME_CASH_LOAN_PURPOSE_PREV_Gasification / water supply,NA

 
 
 <br>

### Exporting the final dataset

In [117]:
data.to_csv('final_data.csv')